# Convert LabelMe Annotation to CSV-dataset

In [2]:
import json
import cv2 as cv
import numpy as np
import csv
import os
from os import walk
import random
from datetime import datetime

In [5]:
# Path the the folder containing the image annotations
path = '../../03 Data/Simple Dataset/'
#path = '../../03 Data/Single_Image/'
validation_percent = 0.2

#print(os.listdir(path))

all_files = os.listdir(path)

file_names = []
for file in all_files: 
    if file[-5:]  == '.json':
        file_names.append(file)
    
#print(file_names)
random.shuffle(file_names)
num_files = len(file_names)

#val_files = file_names[0:int(num_files*validation_percent)]
#train_files = file_names[int(num_files*validation_percent):]

val_file = open('list_of_img_in_val_set_12-03.csv')
val_files = list(val_file)
val_file.close()
for i, img in enumerate(val_files):
    val_files[i] = img.strip('.png\n')+'.json'

train_file = open('list_of_img_in_train_set_12-03.csv')
train_files = list(train_file)
train_file.close()
for i, img in enumerate(train_files):
    train_files[i] = img.strip('.png\n')+'.json'


print(val_files)
print(train_files)

print(len(file_names))
print(len(val_files))
print(len(train_files))

now = datetime.now()
date = now.strftime("%d-%m")

names = ['train_set_'+date+'.csv', 'val_set_'+date+'.csv']
combined_files = [train_files, val_files]
wh_list = []
# create csv of train-set and val-set for feeding to the network
for i in range(len(names)):
    with open(names[i], 'w', newline='') as open_file:
        writer = csv.writer(open_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        with open('list_of_img_in_'+names[i], 'w') as f:
            for file in combined_files[i]:
                with open(path + file) as json_file:
                    data = json.load(json_file)
                    image_name = data['imagePath']
                    f.write("%s\n" % image_name)
                        # check if image contains annotations.
                    if len(data['shapes']) != 0:
                        for annotation in data['shapes']:
                                rot_rect = cv.minAreaRect(np.float32(annotation['points']))
                                bp = cv.boxPoints(rot_rect) # bp = box points
                                #p1 = [rot_rect[0]+]
                                # make sure that width and height are consistent
                                if rot_rect[-2][0] < rot_rect[-2][1]:
                                    rot_rect = (rot_rect[0],(rot_rect[-2][1],rot_rect[-2][0]),rot_rect[-1]+90)
                                # make sure that no angle is above 90 or below -90
                                wh_list.append(list(rot_rect[1]))
                                if rot_rect[-1] > 90:
                                    rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]-180)
                                if rot_rect[-1] < -90:
                                    rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]+180)
                                class_name = annotation['label']
                                writer.writerow([path + image_name, bp[0][0], bp[0][1], bp[1][0], bp[1][1],
                                                 bp[2][0], bp[2][1], bp[3][0], bp[3][1], class_name,rot_rect[-1]])
                                # format: img_path, p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, class_name, angle
                    else:
                        writer.writerow([path + image_name, ",,,,,,,,,,"])
       

['colorIMG_15.json', 'colorIMG_11.json', 'colorIMG_19.json', 'colorIMG_101.json', 'colorIMG_75.json', 'colorIMG_30.json', 'colorIMG_178.json', 'colorIMG_100.json', 'colorIMG_185.json', 'colorIMG_79.json', 'colorIMG_52.json', 'colorIMG_182.json', 'colorIMG_42.json', 'colorIMG_189.json', 'colorIMG_162.json', 'colorIMG_12.json', 'colorIMG_27.json', 'colorIMG_145.json', 'colorIMG_96.json', 'colorIMG_107.json', 'colorIMG_194.json', 'colorIMG_60.json', 'colorIMG_128.json', 'colorIMG_86.json', 'colorIMG_40.json', 'colorIMG_105.json', 'colorIMG_36.json', 'colorIMG_62.json', 'colorIMG_133.json', 'colorIMG_6.json', 'colorIMG_103.json', 'colorIMG_70.json', 'colorIMG_135.json', 'colorIMG_61.json', 'colorIMG_26.json', 'colorIMG_193.json', 'colorIMG_146.json', 'colorIMG_35.json', 'colorIMG_138.json', 'colorIMG_142.json']
['colorIMG_8.json', 'colorIMG_196.json', 'colorIMG_114.json', 'colorIMG_130.json', 'colorIMG_21.json', 'colorIMG_31.json', 'colorIMG_144.json', 'colorIMG_65.json', 'colorIMG_156.jso

In [9]:
print(wh_list)
f = open("brick_wh.txt", 'w')
for width, height in wh_list:
    f.write("%f, %f" % (width, height))

[[102.39974975585938, 25.789684295654297], [102.22373962402344, 28.132707595825195], [102.71240234375, 26.436052322387695], [103.76426696777344, 25.762767791748047], [104.08983612060547, 25.42389678955078], [100.662841796875, 24.508991241455078], [100.84135437011719, 26.136892318725586], [101.40393829345703, 24.442691802978516], [101.52371215820312, 25.768985748291016], [101.5066146850586, 23.431495666503906], [104.1483383178711, 27.15047836303711], [104.07776641845703, 25.773849487304688], [100.67794799804688, 25.084747314453125], [103.07316589355469, 26.78180503845215], [102.03448486328125, 25.080106735229492], [101.43724822998047, 26.10720443725586], [100.87928009033203, 24.063749313354492], [100.87928009033203, 24.063758850097656], [101.67670440673828, 24.063814163208008], [101.19631958007812, 23.75771713256836], [100.43590545654297, 23.13088035583496], [98.91002655029297, 23.905839920043945], [98.91001892089844, 23.13239097595215], [98.41338348388672, 23.366378784179688], [100.291